In [1]:
from graphics import *
import numpy as np
import time

In [2]:
def unitVector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angleBetween(v1, v2, deg = True, debug = False):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::
    """
    if debug:
        print('Первый вектор:', v1)
        print('Второй вектор:', v2)
 
    v1_u = unitVector(v1)
    v2_u = unitVector(v2)
 
    if debug:
        print("v1 ", v1_u, " v2 ", v2_u)
    
    radians = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    result = radians
 
    if debug:
        print('радианы:', result)
    
    if deg:
        result = np.degrees([radians.real])[0]  # переводим в градусы
 
    return result

angleBetween((0, 1, 0), (1, 1, 1))

54.735610317245346

In [3]:
class Model:
    def __init__(self):
        self.vertexes = [(0, 0.5, 0, 1), (0.5, 0, 0, 1), (0, 0, 0.5, 1), (-0.5, 0, 0, 1), (0, 0, -0.5, 1), (0, -0.5, 0, 1)]
        self.edges = [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (2, 3), (3, 4), (4, 1), (5, 1), (5, 2), (5, 3), (5, 4)]

In [10]:
class App:
    def __init__(self, model):
        self.model = model
        self.resolution = (800, 800)
        self.window = GraphWin("Model", self.resolution[0], self.resolution[1], autoflush=True)
        self.window.setBackground('black')
        self.watching_point = (5, 5, 5)
        self.points = []
        self.projectedPoints = []
        self.rotatedPoints = []
        self.modelPoints = []
        self.readModel()
        self.angle = 0
        self.run()
        
    def getViewMatrix(self):
        theta = angleBetween(self.watching_point, (1, 0, 0), False)
        fi = angleBetween(self.watching_point, (0, 0, 1), False)
        ro = np.linalg.norm(self.watching_point)
        return [[-np.sin(theta), -np.cos(fi)*np.cos(theta), -np.sin(fi)*np.cos(theta), 0],
                [np.cos(theta), -np.cos(fi)*np.sin(theta), -np.sin(fi)*np.sin(theta), 0],
                [0, np.sin(fi), -np.cos(fi), 0],
                [0, 0, ro, 1]]

    def getRotationMatrix(self, angle, axis = "x"):
        return [[1, 0, 0, 0],
                [0, np.cos(np.radians(angle)), np.sin(np.radians(angle)), 0],
                [0, -np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0],
                [0, 0, 0, 1]]
    
    def readModel(self):
        for vert in self.model.vertexes:
            vector = np.array(list(vert))
            self.modelPoints.append(vector)
            
    def rotatePoints(self):
#         print("Rotation")
        for point in self.modelPoints:
            result = np.dot(point, self.getRotationMatrix(self.angle))
            self.rotatedPoints.append(result)
    
    def transformPoints(self):
#         print("View Transformation")
        for vert in self.rotatedPoints:
            vector = np.array(list(vert))
            result = np.dot(vector, self.getViewMatrix())
            self.points.append(result)
    
    def projectPoints(self):
#         print("Project")
        ro = np.linalg.norm(self.watching_point)
        for point in self.points:
            x = ro/(2 * point[2]) * point[0]
            y = ro/(2 * point[2]) * point[1]
            self.projectedPoints.append((x, y)) 
    
    def normalizedPoint(self, point, width = 1, height = 1):
        x = (point[0] + width/2)/width
        y = (point[1] + height/2)/height
        return [x, y]
    
    def screenedPoint(self, point, width = 1, height = 1):
        x = point[0] * self.resolution[0]
        y = point[1] * self.resolution[1]
        return [x, y]
        
    def drawEdges(self,edges):
#         print("Edges")
#         print(self.projectedPoints)

        for edge in edges:
            point1 = self.projectedPoints[edge[0]]
            point2 = self.projectedPoints[edge[1]]
        
            point1 = self.normalizedPoint(point1, 1, 1)
            point2 = self.normalizedPoint(point2, 1, 1)
            
            point1 = self.screenedPoint(point1)
            point2 = self.screenedPoint(point2)
            
#             print("Point1 ", point1)
#             print("Point2 ", point2)
            
            line = Line(Point(point1[0], point1[1]), Point(point2[0], point2[1]))
            line.setFill('white')
            line.draw(self.window)
  
    
    def draw(self):
        self.clear()
        self.drawModel()
    
    def clear(self):
#         print("Clean")
        for item in self.window.items[:]:
            item.undraw()
        self.points = []
        self.projectedPoints = []
        self.rotatedPoints = []
        self.window.update()
    
    def drawModel(self):
        self.rotatePoints()
        self.transformPoints()
        self.projectPoints()
        self.drawEdges(self.model.edges)

    def run(self):
        while True:
            self.update()
        
    def update(self):
        self.angle += 5
        self.draw()
        time.sleep(0.1)
#         print("=============")
#         print(self.rotatedPoints)
#         print(self.points)
   

In [11]:
m = Model()
a = App(m)


GraphicsError: Can't draw to closed window

In [ ]:
import numpy as np
import graphics as gr
from time import sleep


def clear(win):
    for item in win.items[:]:
        item.undraw()
    win.update()


def draw(projection, edges):
    gr_points = []
    for p in projection:
        gr_points.append(gr.Point(p[0], p[1]))

    for edge in edges:
        line = gr.Line(gr_points[edge[0] - 1], gr_points[edge[1] - 1])
        line.setFill('white')
        line.draw(win)

def animate(rotated_points, edges):
    projection = np.copy(rotated_points[:, :2])
    projection += 300
    draw(projection, edges)
    sleep(0.1)
    clear(win)

p = np.array([1, 0, 1, 1])

alpha = 5

rotate_x = np.array([[1, 0, 0, 0],
                     [0, np.cos(np.radians(alpha)), np.sin(np.radians(alpha)), 0],
                     [0, -np.sin(np.radians(alpha)), np.cos(np.radians(alpha)), 0],
                     [0, 0, 0, 1]])

rotate_y = np.array([[np.cos(np.radians(alpha)), 0, -np.sin(np.radians(alpha)), 0],
                     [0, 1, 0, 0],
                     [np.sin(np.radians(alpha)), 0, np.cos(np.radians(alpha)), 0],
                     [0, 0, 0, 1]])

rotate_z = np.array([[np.cos(np.radians(alpha)), np.sin(np.radians(alpha)), 0, 0],
                     [-np.sin(np.radians(alpha)), np.cos(np.radians(alpha)), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

p1 = np.array([0, 0, 0, 1])
p2 = np.array([1, 0, 0, 1])
p3 = np.array([1, 1, 0, 1])
p4 = np.array([0, 1, 0, 1])
p5 = np.array([0.5, 0.5, 1, 1])
# p6 = np.array([0, 1, 1, 1])
# p7 = np.array([1, 1, 1, 1])
# p8 = np.array([1, 0, 1, 1])

points = np.array([p1, p2, p3, p4, p5])

edges = np.array([[1, 2], [1, 4], [1, 5],
                  [2, 3], [2, 5], [3, 4],
                  [3, 5], [4, 5]])

points = (points * 2 - 1) / 2
a = 100
points[:, :-1] *= a


win = gr.GraphWin('my_window', 800, 800, autoflush=True)
win.setBackground('black')

rotated_points = np.copy(points)
k = np.random.choice([0, 1, 2])
# while True:
#     iters = 0
#     while k == 0:
#         rotated_points = rotated_points @ rotate_x
#         animate(rotated_points, edges)
#         iters += 1
#         if iters == 15: k = np.random.choice([1, 2]); iters = 0
#     while k == 1:
#         rotated_points = rotated_points @ rotate_y
#         animate(rotated_points, edges)
#         iters += 1
#         if iters == 15: k = np.random.choice([0, 2]); iters = 0
#     while k == 2:
#         rotated_points = rotated_points @ rotate_z
#         animate(rotated_points, edges)
#         iters += 1
#         if iters == 15: k = np.random.choice([0, 1]); iters = 0

d = 400
theta = 20
phi = 70

w_to_e = np.array(
    [[-np.sin(np.radians(theta)), -np.cos(np.radians(phi))*np.cos(np.radians(theta)), -np.sin(np.radians(phi))*np.cos(np.radians(theta)), 0],
     [np.cos(np.radians(theta)), -np.cos(np.radians(phi))*np.sin(np.radians(theta)), -np.sin(np.radians(phi))*np.sin(np.radians(theta)), 0],
     [0, np.sin(np.radians(phi)), -np.cos(np.radians(phi)), 0],
     [0, 0, d, 1]
     ])


#draw((rotated_points @ rotate_x @ rotate_y @ rotate_x @ rotate_y)[:, :2] + 300, edges)

#rotated_points = rotated_points @ rotate_x @ rotate_y @ rotate_x @ rotate_y
#print(rotated_points + 300)
#rotated_points = rotated_points @ w_to_e
#print(rotated_points)
#rotated_points += 300
#print(rotated_points)

#print(rotated_points @ w_to_e)

# x = rotated_points @ w_to_e
# y = x[:, :2] / x[:, 2, np.newaxis] * d / 2
# draw(y+ 300, edges)

#print(rotated_points[:, :2] / rotated_points[:, 2, np.newaxis] / d * 2)# = rotated_points[:, :2] / rotated_points[:, 2]

# ttt = rotated_points[:, :2] / rotated_points[:, 2, np.newaxis]

#ttt += 300
# print(ttt + 300)
# draw(ttt + 300, edges)

while True:
    iters = 0
    while k == 0:
        rotated_points = rotated_points @ rotate_x
        x = np.copy(rotated_points) @ w_to_e
        y = x[:, :2] / x[:, 2, np.newaxis] * d / 2
        draw(y+400, edges)
        sleep(0.1)
        clear(win)
        #animate(rotated_points, edges)
        iters += 1
        if iters == 15: k = np.random.choice([1, 2]); iters = 0
    while k == 1:
        rotated_points = rotated_points @ rotate_y
        x = np.copy(rotated_points) @ w_to_e
        y = x[:, :2] / x[:, 2, np.newaxis] * d / 2
        draw(y+400, edges)
        sleep(0.1)
        clear(win)
        #animate(rotated_points, edges)
        iters += 1
        if iters == 15: k = np.random.choice([0, 2]); iters = 0
    while k == 2:
        rotated_points = rotated_points @ rotate_z
        x = np.copy(rotated_points) @ w_to_e
        y = x[:, :2] / x[:, 2, np.newaxis] * d / 2
        draw(y+400, edges)
        sleep(0.1)
        clear(win)
        #animate(rotated_points, edges)
        iters += 1
        if iters == 15: k = np.random.choice([0, 1]); iters = 0

win.getMouse()
win.close()